In [1]:
import os, sys
sys.path.append("motion_generation")
sys.path.append("rig_agnostic_encoding/functions")
sys.path.append("rig_agnostic_encoding/models")

from motion_generation.MoE import MoE
from motion_generation.MoE_Z import MoE as MoE_Z
import motion_generation
from motion_generation.GRU import GRU
from motion_generation.LSTM import LSTM
from motion_generation.MotionGeneration import MotionGenerationModel
from motion_generation.MotionGeneration_v2 import MotionGenerationModel as MotionGenerationModel_v2
from motion_generation.MotionGeneration_v3 import MotionGenerationModel as MotionGenerationModel_v3
from motion_generation.MotionGeneration_v4 import MotionGenerationModel as MotionGenerationModel_v4
from motion_generation.MotionGeneration_v4_Concat import MotionGenerationModel as MotionGenerationModel_Concat
from motion_generation.MotionGenerationVAE_v4 import MotionGenerationModel as MotionGenerationModel_VAE
from motion_generation.MotionGenerationVAE_Z import MotionGenerationModel as MotionGenerationModel_VAE_ZZ
from motion_generation.MotionGenerationVAE_Z_everywhere import MotionGenerationModel as MotionGenerationModel_VAE_Z
from motion_generation.MotionGenerationRNN import MotionGenerationModelRNN
from motion_generation.MotionGenerationBatch import MotionGenerationModelBatch
from rig_agnostic_encoding.models.MLP import MLP
from rig_agnostic_encoding.models.MLP_v2 import MLP as MLP_v2
from rig_agnostic_encoding.models.MLP_v3 import MLP as MLP_v3

from rig_agnostic_encoding.models.DEC import DEC
from rig_agnostic_encoding.models.MLP_MIX import MLP_MIX
from rig_agnostic_encoding.models.MLP_MIX_cosine import MLP_MIX as MLP_MIX_COSINE

from rig_agnostic_encoding.models.RBF import RBF
import rig_agnostic_encoding.models.RBF as RB
from rig_agnostic_encoding.models.RBF_Ortho import RBF as RBF_ORTHO
from rig_agnostic_encoding.models.RBF_Ortho_v2 import RBF as RBF_ORTHO_v2


from rig_agnostic_encoding.models.VAE import VAE
from rig_agnostic_encoding.functions.DataProcessingFunctions import clean_checkpoints
from GlobalSettings import MODEL_PATH
import bz2
from cytoolz import concat, sliding_window, accumulate
from operator import add
from collections import OrderedDict
import torch
import torch.nn as nn
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import func as F
import _pickle as pickle
import json as js
import importlib
import random

In [74]:
config = {
    "hidden_dim": 256,
    "k": 256,
    "z_dim": 256,
    "lr": 1e-4,
    "batch_size": 16,
    "keep_prob": 0,
    "loss_fn":torch.nn.functional.mse_loss,
    "optimizer":torch.optim.AdamW,
    "scheduler":torch.optim.lr_scheduler.StepLR,
    "scheduler_param": {"step_size":80, "gamma":.9},
    "basis_func":"gaussian",
    "n_centroid":64,
    "k_experts": 4,
    "gate_size": 128,
    "g_hidden_dim": 512,
    "num_layers": 4,
    "autoregress_prob":0.3,
    "autoregress_inc":20,
    "autoregress_ep":20,
    "autoregress_max_prob":1,
    "cost_hidden_dim":128,
    "seq_len":13,
    "device":"cuda"
    }

In [5]:
def getFilesNames(file_paths, data_path, MAX_FILES=-1):
    for dname, dirs, files in os.walk(data_path):
        for i, file in enumerate(files):
            file_paths.append(os.path.join(dname, file))
            if MAX_FILES > 0 and i >= MAX_FILES:
                break
    return file_paths

data_path = "/home/nuoc/Documents/MEX/data/data/Dataset_R1_Two_1"
data_path2 = "/home/nuoc/Documents/MEX/data/data/Dataset2_R5_Two_1"
# data_path3 = "/home/nuoc/Documents/MEX/data/data/Dataset_R3_Two_1"
# data_path4 = "/home/nuoc/Documents/MEX/data/data/Dataset_R4_Two_1"
file_paths = getFilesNames([],data_path)
file_paths2 = getFilesNames([],data_path2)
# file_paths3 = getFilesNames([],data_path3)
# file_paths4 = getFilesNames([],data_path4)

print(len(file_paths))
print(len(file_paths2))





240
240


In [3]:
phase_features = ["phase_vec_l2"]
pose_features = ["pos", "rotMat2", "velocity"]
cost_features = ["posCost", "rotCost"]
pose_label_feature = ["chainPos", "isLeft", "geoDistanceNormalised"]
target_features = ["targetPosition", "targetRotation"]
features = phase_features + pose_features + cost_features + target_features
clips = []
feature_dims = {}


In [7]:
data = F.process_data_multithread(file_paths, features)
data2 = F.process_data_multithread(file_paths2, features)

2021-05-10 08:59:03,211	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-10 08:59:24,517	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


In [8]:
pose_labels1 = F.process_data_multithread([file_paths[0]], pose_label_feature)
pose_labels2 = F.process_data_multithread([file_paths2[0]], pose_label_feature)

2021-05-10 08:59:45,646	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-10 08:59:50,603	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


In [9]:
obj = {"data":data, "data2":data2, "pose_label1":pose_labels1, "pose_label2":pose_labels2}
F.save(obj, filename="transfer_learning_set_R1-R5_Two", path="/home/nuoc/Documents/MEX/data/")

In [4]:
obj = F.load("/home/nuoc/Documents/MEX/data/transfer_learning_set_R1-R5_Two.pbz2")
data = obj["data"]
data2 = obj["data2"]

In [5]:
pose_labels1 = obj["pose_label1"]
pose_labels2 = obj["pose_label2"]

In [6]:
feature_dims = data[0][1]
feature_dims2 = data2[0][1]
clips = [np.copy(d[0]) for d in data]
clips2 = [np.copy(d[0]) for d in data2]

pose_labels1 = pose_labels1[0]
pose_labels2 = pose_labels2[0]

In [7]:
phase_dim = sum([feature_dims[feature] for feature in phase_features])
pose_dim = sum([feature_dims[feature] for feature in pose_features])
pose_dim2 = sum([feature_dims2[feature] for feature in pose_features])
cost_dim = sum([feature_dims[feature] for feature in cost_features])
target_dim = sum([feature_dims[feature] for feature in target_features])
print(phase_dim, " ", cost_dim, " ", target_dim)

8   24   48


In [8]:
x_tensors = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips])
y_tensors = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips])

x_tensors2 = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips2])
y_tensors2 = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips2])


pose_data1 = x_tensors[:,  :,  phase_dim:phase_dim+pose_dim]
pose_data2 = x_tensors2[:, :, phase_dim:phase_dim+pose_dim2]
pose_data = torch.cat((pose_data1, pose_data2), dim=2)

In [9]:
dataset_p = TensorDataset(pose_data, pose_data)
dataset_p1 = TensorDataset(pose_data1, pose_data1)
dataset_p2 = TensorDataset(pose_data2, pose_data2)
datasetR1 = TensorDataset(x_tensors, y_tensors)
datasetR2 = TensorDataset(x_tensors2, y_tensors2)
# datasetR3 = TensorDataset(x_tensors3, y_tensors3)
# datasetR4 = TensorDataset(x_tensors4, y_tensors4)
# datasetR1_R3 = TensorDataset(x_tensors, y_tensors3)

N = len(x_tensors)

train_ratio = int(.7*N)
val_ratio = int((N-train_ratio) / 2.0)
test_ratio = N - train_ratio - val_ratio

train_set_p, val_set_p, test_set_p = random_split(dataset_p, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_set_p1, val_set_p1, test_set_p1 = random_split(dataset_p1, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_set_p2, val_set_p2, test_set_p2 = random_split(dataset_p2, [val_ratio*2, val_ratio, train_ratio-test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR1, val_setR1, test_setR1 = random_split(datasetR1, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
train_setR2, val_setR2, test_setR2 = random_split(datasetR2, [val_ratio*2, val_ratio, train_ratio-val_ratio], generator=torch.Generator().manual_seed(2021))
# train_setR3, val_setR3, test_setR3 = random_split(datasetR3, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
# train_setR4, val_setR4, test_setR4 = random_split(datasetR4, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
# train_setR1_3, val_setR1_3, test_setR1_3 = random_split(datasetR1_R3, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
# print(len(train_set), len(val_set), len(test_set))
# print(len(train_set2), len(val_set2), len(test_set2))

In [10]:
def extract_targets(train_set, val_set, test_set, target_dim):
    t1, t2, t3, t33 = [], [], [], []
    for i in range(len(train_set)):
        x = train_set[i][0]
        y = train_set[i][1]
        t1.append((x[:, :-target_dim], y[:, :-target_dim]))
    for i in range(len(val_set)):
        x = val_set[i][0]
        y = val_set[i][1]
        t2.append((x[:, :-target_dim:], y[:, :-target_dim:]))
    for i in range(len(test_set)):
        x = test_set[i][0]
        y = test_set[i][1]
        t3.append((x[:, :-target_dim:], y[:, :-target_dim:]))
        t33.append((x[:, :-target_dim:], y))
    return t1, t2, t3, t33




In [11]:
t1, v1, te1, te11 = extract_targets(train_set=train_setR1, val_set=val_setR1, test_set=test_setR1, target_dim=target_dim)
t2, v2, te2, te22 = extract_targets(train_set=train_setR2, val_set=val_setR2, test_set=test_setR2, target_dim=target_dim)

# AE ONLY

In [44]:
import pytorch_lightning as pl
import numpy as np
from cytoolz import sliding_window
from collections import OrderedDict
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import sys, os
import bz2
import _pickle as pickle
sys.path.append("../../")
from GlobalSettings import DATA_PATH, MODEL_PATH

class MLP(pl.LightningModule):
    def __init__(self, config:dict=None, dimensions:list=None, pose_labels=None,
                 train_set=None, val_set=None, test_set=None, pos_dim=0, rot_dim=0, vel_dim=0,
                 name:str="model", single_module:int=0, save_period=5,
                 workers=6):

        super(MLP, self).__init__()
        self.name = name
        self.dimensions = dimensions
        self.single_module = single_module
        self.act = nn.ELU
        self.save_period = save_period
        self.workers = workers
        self.config=config

        self.pos_dim = pos_dim
        self.rot_dim = pos_dim + rot_dim
        self.vel_dim = self.rot_dim + vel_dim

        self.hidden_dim = config["hidden_dim"]
        self.keep_prob = config["keep_prob"]
        self.k = config["k"]
        self.learning_rate = config["lr"]
        self.batch_size = config["batch_size"]

        self.dimensions = dimensions if len(dimensions) > 1 else \
            [dimensions[0], self.hidden_dim, self.hidden_dim, self.k]

        self.loss_fn = config["loss_fn"] if "loss_fn" in config else nn.functional.mse_loss
        self.opt = config["optimizer"] if "optimizer" in config else torch.optim.Adam
        self.scheduler = config["scheduler"] if "scheduler" in config else None
        self.scheduler_param = config["scheduler_param"] if "scheduler_param" in config else None

        self.pose_labels = pose_labels  # should be Tensor(1,63) for example
        self.use_label = pose_labels is not None

        self.train_set, self.val_set, self.test_set = train_set, val_set, test_set
        self.best_val_loss = np.inf

        self.encoder, self.decoder = nn.Module(), nn.Module()
        self.build()
        if "device" not in config:
            config["device"] = "cuda"

        self.encoder.to(config["device"])
        self.decoder.to(config["device"])

        self.encoder.apply(self.init_params)
        self.decoder.apply(self.init_params)

    def build(self):
        layer_sizes = list(sliding_window(2, self.dimensions))
        if self.single_module == -1 or self.single_module == 0:
            layers = []
            for i, size in enumerate(layer_sizes):
                layers.append(("fc"+str(i), nn.Linear(size[0], size[1])))
                if i < len(self.dimensions)-2:
                    layers.append(("act"+str(i), self.act()))
                if i == 0:
                    layers.append(("drop", nn.Dropout(self.keep_prob)))
            self.encoder = nn.Sequential(OrderedDict(layers))
        else:
            self.encoder = nn.Sequential()

        if self.single_module == 0 or self.single_module == 1:
            layers = []
            if self.pose_labels is not None:
                layer_sizes[-1] = (layer_sizes[-1][0], layer_sizes[-1][1] + self.pose_labels.numel())
            for i, size in enumerate(layer_sizes[-1::-1]):
                layers.append(("fc"+str(i), nn.Linear(size[1], size[0])))
                if i < len(self.dimensions)-2:
                    layers.append(("act"+str(i), self.act()))
                # if i == 0:
                    # layers.append(("drop", nn.Dropout(self.keep_prob)))
            self.decoder = nn.Sequential(OrderedDict(layers))
        else:
            self.decoder = nn.Sequential()


    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.decode(self.encode(x))

    def encode(self, x):
        return self.encoder(x)

    def decode(self, h):
        if self.use_label:
            h = torch.cat((h, self.pose_labels.expand(h.shape[0],-1)), dim=1)
        return self.decoder(h)

    def decode_label(self, h):
        h = torch.cat((h, self.pose_labels.expand(h.shape[0],-1)), dim=1)
        return self.decoder(h)

    def loss(self, x, y):
        recon_loss = self.loss_fn(x, y)

        px, py =x[:, :self.pos_dim].detach(), y[:, :self.pos_dim].detach()
        px_norm, py_norm = torch.sum(px**2), torch.sum(py**2)
        pos_loss = torch.mean((px-py)**2 / torch.sqrt(py_norm))

        rx, ry = x[:, self.pos_dim:self.rot_dim].detach() % 2 * np.pi, y[:, self.pos_dim:self.rot_dim].detach() % 2 * np.pi
        rot_loss = self.loss_fn(rx, ry)

        return recon_loss, pos_loss, rot_loss

    def training_step(self, batch, batch_idx):
        x, y = batch
        prediction = self(x)
        recon_loss, pos_loss, rot_loss = self.loss(prediction, y)

        self.log("ptl/train_loss", recon_loss)
        self.log("ptl/train_pos_loss", pos_loss)
        self.log("ptl/train_rot_loss", rot_loss)
        return recon_loss

    def validation_step(self, batch, batch_idx):
        x, y = batch

        prediction = self(x)
        recon_loss, pos_loss, rot_loss = self.loss(prediction, y)

        self.log("ptl/val_loss", recon_loss, prog_bar=True)
        self.log("ptl/val_pos_loss", pos_loss, prog_bar=True)
        self.log("ptl/val_rot_loss", rot_loss, prog_bar=True)
        return {"val_loss":recon_loss}

    def test_step(self, batch, batch_idx):
        x, y = batch

        prediction = self(x)
        recon_loss, pos_loss, rot_loss = self.loss(prediction, y)

        self.log("ptl/test_loss", recon_loss)
        self.log("ptl/test_pos_loss", pos_loss)
        self.log("ptl/test_rot_loss", rot_loss)
        return {"test_loss":recon_loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)
        if avg_loss < self.best_val_loss:
            self.best_val_loss = avg_loss
            if self.current_epoch % self.save_period == 0:
                self.save_checkpoint(best_val_loss=self.best_val_loss.item())

    def save_checkpoint(self, best_val_loss:float=np.inf, checkpoint_dir=MODEL_PATH):
        config = {
            "hidden_dim":self.hidden_dim,
            "k":self.k,
            "lr":self.learning_rate,
            "batch_size":self.batch_size,
            "keep_prob":self.keep_prob,
            "optimizer":self.opt,
            "scheduler":self.scheduler,
            "scheduler_param":self.scheduler_param,
            "device":self.config["device"],
            "dims": [self.pos_dim, self.rot_dim, self.vel_dim]
        }
        model = {"config":config, "name":self.name,"dimensions":self.dimensions,
                 "pose_labels": self.pose_labels,
                 "single_module":self.single_module,
                 "encoder":self.encoder.state_dict(),
                 "decoder":self.decoder.state_dict()}

        if not os.path.exists(checkpoint_dir):
            os.mkdir(checkpoint_dir)
        path = os.path.join(checkpoint_dir, self.name)
        if not os.path.exists(path):
            os.mkdir(path)

        filePath = os.path.join(path, str(best_val_loss)+"."+str(self.k)+".pbz2")
        with bz2.BZ2File(filePath, "w") as f:
            pickle.dump(model, f)
        return filePath

    @staticmethod
    def load_checkpoint(filePath):
        with bz2.BZ2File(filePath, "rb") as f:
            obj = pickle.load(f)
        model = MLP(config=obj["config"], single_module=obj["single_module"], pose_labels=obj["pose_labels"],
                    name=obj["name"], dimensions=obj["dimensions"])

        model.encoder.load_state_dict(obj["encoder"])
        model.decoder.load_state_dict(obj["decoder"])
        model.pos_dim = obj["config"]["dims"][0]
        model.rot_dim = obj["config"]["dims"][1]
        model.vel_dim = obj["config"]["dims"][2]
        return model

    def freeze(self, flag=False):
        self.encoder.requires_grad_(flag)
        self.decoder.requires_grad_(flag)

    def configure_optimizers(self):
        optimizer = self.opt(self.parameters(), lr=self.learning_rate)
        if self.scheduler is not None:
            scheduler = self.scheduler(optimizer, **self.scheduler_param)
            return [optimizer], [scheduler]
        return optimizer

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    @staticmethod
    def init_params(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(.01)


In [45]:
ae_name1 = "AE_R1_Two_v3"
ae1 = MLP(config=config, dimensions=[pose_dim],
                 pos_dim=feature_dims["pos"], rot_dim=feature_dims["rotMat2"], vel_dim=feature_dims["velocity"],
                 train_set=train_set_p1, val_set=val_set_p1, test_set=test_set_p1, name=ae_name1)

# ae1 = MLP.load_checkpoint("/home/nuoc/Documents/MEX/models/version_0.2/AE_R1_Two/0.001.256.pbz2")
# ae1.name = ae_name1

In [60]:
ae_name2 = "AE_R5_Two"
ae2 = MLP(config=config, dimensions=[pose_dim2],
                 pos_dim=feature_dims2["pos"], rot_dim=feature_dims2["rotMat2"], vel_dim=feature_dims2["velocity"],
                 train_set=train_set_p2, val_set=val_set_p2, test_set=test_set_p2, name=ae_name2)
#
# ae2 = MLP.load_checkpoint("/home/nuoc/Documents/MEX/models/version_0.2/AE_R5_Two/0.001.256.pbz2")
# ae2.name = ae_name2

In [13]:
def fit(model, name, version="0.1", MAX_EPOCHS=200):
    logger=TensorBoardLogger(save_dir="logs/", name=name, version=version)

    trainer = pl.Trainer(
        default_root_dir="/home/nuoc/Documents/MEX/src/motion_generation/checkpoints",
        gpus=1, precision=16,
        min_epochs=20,
        logger=logger,
        max_epochs=MAX_EPOCHS,
        stochastic_weight_avg=True
    )
    trainer.fit(model)
    trainer.test(model)

In [46]:
fit(ae1, ae_name1)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 227 K 
1 | decoder | Sequential | 227 K 
---------------------------------------
454 K     Trainable params
0         Non-trainable params
454 K     Total params
1.817     Total estimated model params size (MB)


Traceback (most recent call last):
  File "/home/nuoc/miniconda3/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
    close()
  File "/home/nuoc/miniconda3/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/nuoc/miniconda3/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/home/nuoc/miniconda3/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
    close()
  File "/home/nuoc/miniconda3/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/nuoc/miniconda3/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f835ce6e3a0> for <torch.optim.swa_utils.SWALR object at 0x7f835cd21850>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.018363110721111298,
 'ptl/test_pos_loss': 3.75769886886701e-05,
 'ptl/test_rot_loss': 0.8903936743736267,
 'test_loss': 0.021002743393182755}
--------------------------------------------------------------------------------


In [61]:
fit(ae2, ae_name2)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 230 K 
1 | decoder | Sequential | 230 K 
---------------------------------------
460 K     Trainable params
0         Non-trainable params
460 K     Total params
1.842     Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f835d0daee0> for <torch.optim.swa_utils.SWALR object at 0x7f835cf2d130>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.04999605566263199,
 'ptl/test_pos_loss': 9.325029532192275e-05,
 'ptl/test_rot_loss': 5.256495475769043,
 'test_loss': 0.06917048245668411}
--------------------------------------------------------------------------------


In [62]:
clean_checkpoints(os.path.join(MODEL_PATH,ae_name1))
clean_checkpoints(os.path.join(MODEL_PATH,ae_name2))

ae1.save_checkpoint(best_val_loss=0.001)
ae2.save_checkpoint(best_val_loss=0.001)


'/home/nuoc/Documents/MEX/models/version_0.2/AE_R5_Two/0.001.256.pbz2'

In [52]:

class MotionGenerationModel(pl.LightningModule):
    def __init__(self, config:dict=None, Model=None, pose_autoencoder=None, middle_layer=None, feature_dims=None,
                 input_slicers:list=None, output_slicers:list=None,
                 train_set=None, val_set=None, test_set=None, name="MotionGeneration", workers=8):
        super().__init__()

        self.feature_dims = feature_dims
        self.config=config
        self.workers = workers

        self.loss_fn = config["loss_fn"] if "loss_fn" in config else nn.functional.mse_loss
        self.opt = config["optimizer"] if "optimizer" in config else torch.optim.Adam
        self.scheduler = config["scheduler"] if "scheduler" in config else None
        self.scheduler_param = config["scheduler_param"] if "scheduler_param" in config else None
        self.batch_size = config["batch_size"]
        self.learning_rate = config["lr"]
        self.seq_len = config["seq_len"]

        self.autoregress_prob = config["autoregress_prob"]
        self.autoregress_inc = config["autoregress_inc"]
        self.autoregress_ep = config["autoregress_ep"]
        self.autoregress_max_prob = config["autoregress_max_prob"]

        self.best_val_loss = np.inf
        self.phase_smooth_factor = 0.9

        self.pose_autoencoder = pose_autoencoder if pose_autoencoder is not None else \
            MLP_v2(config=config, dimensions=[feature_dims["pose_dim"]], name="PoseAE")
        self.middle_layer = middle_layer if middle_layer is not None else \
            nn.Sequential(nn.Linear(in_features=self.pose_autoencoder.dimensions[-1], out_features=self.pose_autoencoder.dimensions[-1]))
        self.use_label = pose_autoencoder is not None and pose_autoencoder.use_label

        cost_hidden_dim = config["cost_hidden_dim"]
        self.cost_encoder = MLP_v2(config=config,
                            dimensions=[feature_dims["cost_dim"], cost_hidden_dim, cost_hidden_dim, cost_hidden_dim],
                            name="CostEncoder", single_module=-1)

        self.generationModel =  Model(config=config,
                                      dimensions=[feature_dims["g_input_dim"], feature_dims["g_output_dim"]],
                                      phase_input_dim = feature_dims["phase_dim"])


        self.input_dims = input_slicers
        self.output_dims = output_slicers
        self.in_slices = [0] + list(accumulate(add, input_slicers))
        self.out_slices = [0] + list(accumulate(add, output_slicers))

        self.phase_dim = feature_dims["phase_dim"]
        self.posCost_dim = self.in_slices[-2]+feature_dims["posCost"]
        self.rotCost_dim = self.posCost_dim + feature_dims["rotCost"]
        self.pos_dim = feature_dims["pos_dim"]
        self.rot_dim = self.phase_dim + feature_dims["pos_dim"] + feature_dims["rot_dim"]
        self.vel_dim = self.rot_dim + feature_dims["vel_dim"]

        self.train_set = train_set
        self.val_set = val_set
        self.test_set = test_set
        self.name = name

        self.automatic_optimization = False

    def forward(self, x):
        x_tensors = [x[:, d0:d1] for d0, d1 in zip(self.in_slices[:-1], self.in_slices[1:])]

        pose_h = self.middle_layer(self.pose_autoencoder.encode(x_tensors[1]))
        embedding = torch.cat([pose_h, self.cost_encoder(x_tensors[2])], dim=1)
        out = self.generationModel(embedding, x_tensors[0])
        out_tensors = [out[:, d0:d1] for d0, d1 in zip(self.out_slices[:-1], self.out_slices[1:])]
        phase = self.update_phase(x_tensors[0], out_tensors[0])
        new_pose = self.pose_autoencoder.decode(out_tensors[1])

        return [phase, new_pose, out_tensors[-1]]

    def loss(self, px, py, rx, ry):
        px_norm, py_norm = torch.sum(px**2), torch.sum(py**2)
        pos_loss = torch.mean((px-py)**2 / torch.sqrt(py_norm))

        rx2, ry2 = rx % 2 * np.pi, ry % 2 * np.pi
        rot_loss = self.loss_fn(rx2, ry2)

        return pos_loss, rot_loss

    def step(self, x, y):
        opt = self.optimizers()

        n = x.size()[1]
        tot_loss = 0
        tot_pos_loss = 0
        tot_rot_loss = 0

        min_pos_cost = 100
        min_target_pos_cost = 100
        min_rot_cost = 100
        min_target_rot_cost = 100

        sum_pos_cost = 0
        sum_target_pos_cost = 0
        sum_rot_cost = 0
        sum_target_rot_cost = 0

        x_c = x[:,0,:]
        autoregress_bools = torch.randn(n) < self.autoregress_prob
        for i in range(1, n):
            y_c = y[:,i-1,:]

            out= self(x_c)
            recon = torch.cat(out, dim=1)

            pos_x = recon[:, self.phase_dim:self.pos_dim]
            pos_y = y_c[:, self.phase_dim:self.pos_dim]

            rot_x = recon[:, self.pos_dim:self.rot_dim]
            rot_y = y_c[:, self.pos_dim:self.rot_dim]

            pos_cost_x = torch.mean(recon[:, self.in_slices[-2]:self.posCost_dim])
            rot_cost_x = torch.mean(recon[:, self.posCost_dim:])

            pos_cost_y = torch.mean(y_c[:, self.in_slices[-2]:self.posCost_dim])
            rot_cost_y = torch.mean(y_c[:, self.posCost_dim:])

            loss = self.loss_fn(recon, y_c)
            pos_loss, rot_loss = self.loss(pos_x, pos_y, rot_x, rot_y)

            tot_loss += loss.detach()
            tot_pos_loss += pos_loss.detach()
            tot_rot_loss += rot_loss.detach()

            min_pos_cost = min(pos_cost_x, min_pos_cost)
            min_target_pos_cost = min(pos_cost_y, min_target_pos_cost)
            min_rot_cost = min(rot_cost_x, min_rot_cost)
            min_target_rot_cost = min(rot_cost_y, min_target_rot_cost)

            sum_pos_cost += pos_cost_x
            sum_target_pos_cost += pos_cost_y
            sum_rot_cost += rot_cost_x
            sum_target_rot_cost += rot_cost_y

            opt.zero_grad()
            self.manual_backward(loss)
            opt.step()

            if autoregress_bools[i]:
                x_c = torch.cat(out,dim=1).detach()
            else:
                x_c = x[:,i,:]

        tot_loss /= float(n)
        tot_pos_loss /= float(n)
        tot_rot_loss /= float(n)
        return tot_loss, tot_pos_loss, tot_rot_loss,\
               min_pos_cost, min_rot_cost, min_target_pos_cost, min_target_rot_cost,\
               sum_pos_cost, sum_rot_cost, sum_target_pos_cost, sum_target_rot_cost

    def step_eval(self, x, y):
        n = x.size()[1]
        tot_loss = 0
        tot_pos_loss = 0
        tot_rot_loss = 0

        min_pos_cost = 100
        min_target_pos_cost = 100
        min_rot_cost = 100
        min_target_rot_cost = 100

        sum_pos_cost = 0
        sum_target_pos_cost = 0
        sum_rot_cost = 0
        sum_target_rot_cost = 0

        x_c = x[:,0,:]
        autoregress_bools = torch.randn(n) < self.autoregress_prob
        for i in range(1, n):
            y_c = y[:,i-1,:]

            out= self(x_c)
            recon = torch.cat(out, dim=1)
            loss = self.loss_fn(recon, y_c)
            tot_loss += loss.detach()

            pos_x = recon[:, self.phase_dim:self.pos_dim]
            pos_y = y_c[:, self.phase_dim:self.pos_dim]

            rot_x = recon[:, self.pos_dim:self.rot_dim]
            rot_y = y_c[:, self.pos_dim:self.rot_dim]

            pos_cost_x = torch.mean(recon[:, self.in_slices[-2]:self.posCost_dim])
            rot_cost_x = torch.mean(recon[:, self.posCost_dim:])

            pos_cost_y = torch.mean(y_c[:, self.in_slices[-2]:self.posCost_dim])
            rot_cost_y = torch.mean(y_c[:, self.posCost_dim:])

            pos_loss, rot_loss = self.loss(pos_x, pos_y, rot_x, rot_y)

            tot_pos_loss += pos_loss.detach()
            tot_rot_loss += rot_loss.detach()

            min_pos_cost = min(pos_cost_x, min_pos_cost)
            min_target_pos_cost = min(pos_cost_y, min_target_pos_cost)
            min_rot_cost = min(rot_cost_x, min_rot_cost)
            min_target_rot_cost = min(rot_cost_y, min_target_rot_cost)

            sum_pos_cost += pos_cost_x
            sum_target_pos_cost += pos_cost_y
            sum_rot_cost += rot_cost_x
            sum_target_rot_cost += rot_cost_y

            if autoregress_bools[i]:
                x_c = torch.cat(out,dim=1).detach()
            else:
                x_c = x[:,i,:]

        tot_loss /= float(n)
        tot_pos_loss /= float(n)
        tot_rot_loss /= float(n)
        return tot_loss, tot_pos_loss, tot_rot_loss,\
               min_pos_cost, min_rot_cost, min_target_pos_cost, min_target_rot_cost,\
               sum_pos_cost, sum_rot_cost, sum_target_pos_cost, sum_target_rot_cost

    def training_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, self.seq_len, x.shape[-1])
        y = y.view(-1, self.seq_len, y.shape[-1])
        loss, tot_pos_loss, tot_rot_loss,\
               min_pos_cost, min_rot_cost, min_target_pos_cost, min_target_rot_cost,\
               sum_pos_cost, sum_rot_cost, sum_target_pos_cost, sum_target_rot_cost = self.step(x,y)

        self.log("ptl/train_loss", loss, prog_bar=True)
        self.log("ptl/train_pos_loss", tot_pos_loss)
        self.log("ptl/train_rot_loss", tot_rot_loss)

        self.log("ptl/train_min_pos_cost", min_pos_cost)
        self.log("ptl/train_min_target_pos_cost", min_target_pos_cost)
        self.log("ptl/train_min_rot_cost", min_rot_cost)
        self.log("ptl/train_min_target_rot_cost", min_target_rot_cost)

        self.log("ptl/train_sum_pos_cost", sum_pos_cost)
        self.log("ptl/train_sum_target_pos_cost", sum_target_pos_cost)
        self.log("ptl/train_sum_rot_cost", sum_rot_cost)
        self.log("ptl/train_sum_target_rot_cost", sum_target_rot_cost)

    def validation_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, self.seq_len, x.shape[-1])
        y = y.view(-1, self.seq_len, y.shape[-1])
        loss, tot_pos_loss, tot_rot_loss,\
               min_pos_cost, min_rot_cost, min_target_pos_cost, min_target_rot_cost,\
               sum_pos_cost, sum_rot_cost, sum_target_pos_cost, sum_target_rot_cost = self.step_eval(x,y)

        self.log("ptl/val_loss", loss, prog_bar=True)
        self.log("ptl/val_pos_loss", tot_pos_loss,prog_bar=True)
        self.log("ptl/val_rot_loss", tot_rot_loss,prog_bar=True)

        self.log("ptl/val_min_pos_cost", min_pos_cost)
        self.log("ptl/val_min_target_pos_cost", min_target_pos_cost)
        self.log("ptl/val_min_rot_cost", min_rot_cost)
        self.log("ptl/val_min_target_rot_cost", min_target_rot_cost)

        self.log("ptl/val_sum_pos_cost", sum_pos_cost)
        self.log("ptl/val_sum_target_pos_cost", sum_target_pos_cost)
        self.log("ptl/val_sum_rot_cost", sum_rot_cost)
        self.log("ptl/val_sum_target_rot_cost", sum_target_rot_cost)

        return {"val_loss":loss}

    def test_step(self, batch, batch_idx):
        x, y = batch

        x = x.view(-1, self.seq_len, x.shape[-1])
        y = y.view(-1, self.seq_len, y.shape[-1])
        loss, tot_pos_loss, tot_rot_loss,\
               min_pos_cost, min_rot_cost, min_target_pos_cost, min_target_rot_cost,\
               sum_pos_cost, sum_rot_cost, sum_target_pos_cost, sum_target_rot_cost = self.step_eval(x,y)
        self.log("ptl/test_loss", loss, prog_bar=True)
        self.log("ptl/test_pos_loss", tot_pos_loss,prog_bar=True)
        self.log("ptl/test_rot_loss", tot_rot_loss,prog_bar=True)

        self.log("ptl/test_min_pos_cost", min_pos_cost,prog_bar=True)
        self.log("ptl/test_min_target_pos_cost", min_target_pos_cost,prog_bar=True)
        self.log("ptl/test_min_rot_cost", min_rot_cost,prog_bar=True)
        self.log("ptl/test_min_target_rot_cost", min_target_rot_cost,prog_bar=True)

        self.log("ptl/test_sum_pos_cost", sum_pos_cost,prog_bar=True)
        self.log("ptl/test_sum_target_pos_cost", sum_target_pos_cost,prog_bar=True)
        self.log("ptl/test_sum_rot_cost", sum_rot_cost,prog_bar=True)
        self.log("ptl/test_sum_target_rot_cost", sum_target_rot_cost,prog_bar=True)

        return {"test_loss":loss}

    def validation_epoch_end(self, outputs):
        if self.current_epoch > 0 and self.current_epoch % self.autoregress_ep == 0:
            self.autoregress_prob = min(self.autoregress_max_prob, self.autoregress_prob+self.autoregress_inc)

        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)
        if avg_loss < self.best_val_loss:
            self.best_val_loss = avg_loss
            self.save_checkpoint(best_val_loss=self.best_val_loss.item())

    def save_checkpoint(self, best_val_loss=np.inf, checkpoint_dir=MODEL_PATH):
        path = os.path.join(checkpoint_dir, self.name)

        config = {}
        for key, val in self.config.items():
            config[key] = val

        pose_autoencoder_path = self.pose_autoencoder.save_checkpoint(best_val_loss=best_val_loss, checkpoint_dir=path)
        cost_encoder_path = self.cost_encoder.save_checkpoint(best_val_loss=best_val_loss, checkpoint_dir=path)
        generationModel_path = self.generationModel.save_checkpoint(best_val_loss=best_val_loss, checkpoint_dir=path)

        model = {"name":self.name,
                 "config":config,
                 "pose_autoencoder_path":pose_autoencoder_path,
                 "cost_encoder_path": cost_encoder_path,
                 "motionGenerationModelPath":generationModel_path,
                 "in_slices":self.in_slices,
                 "out_slices":self.out_slices,
                 "feature_dims":self.feature_dims,
                 "middle_layer_dict":self.middle_layer.state_dict(),
                 }

        if not os.path.exists(path):
            os.mkdir(path)
        with bz2.BZ2File(os.path.join(path,
                                      str(best_val_loss)+".pbz2"), "w") as f:
            pickle.dump(model, f)

    @staticmethod
    def load_checkpoint(filename, Model, MiddleModel:nn.Module=None):
        with bz2.BZ2File(filename, "rb") as f:
            obj = pickle.load(f)

        pose_autoencoder = MLP_v2.load_checkpoint(obj["pose_autoencoder_path"])
        cost_encoder = MLP_v2.load_checkpoint(obj["cost_encoder_path"])
        generationModel = Model.load_checkpoint(obj["motionGenerationModelPath"])

        model = MotionGenerationModel(config=obj["config"], feature_dims=obj["feature_dims"], Model=Model,
                                      input_slicers=obj["in_slices"], output_slicers=obj["out_slices"],
                                      name=obj["name"])
        if MiddleModel is None:
            MiddleModel = nn.Linear(in_features=pose_autoencoder.dimensions[-1], out_features=pose_autoencoder.dimensions[-1])

        MiddleModel.load_state_dict(obj["middle_layer_dict"])
        model.in_slices = obj["in_slices"]
        model.out_slices = obj["out_slices"]
        model.pose_autoencoder = pose_autoencoder
        model.cost_encoder = cost_encoder
        model.generationModel = generationModel

        return model

    def update_phase(self, p1, p2):
        return self.phase_smooth_factor * p2 + (1-self.phase_smooth_factor)*p1

    def configure_optimizers(self):
        optimizer = self.opt(self.parameters(), lr=self.learning_rate)
        if self.scheduler is not None:
            scheduler = self.scheduler(optimizer, **self.scheduler_param)
            return [optimizer], [scheduler]
        return optimizer

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, pin_memory=True, num_workers=self.workers)

    def swap_pose_encoder(self, pose_encoder=None,
                          input_dim=None, output_dim=None,
                          feature_dims=None, freeze=False):
        self.pose_autoencoder = pose_encoder
        self.input_dims = input_dim
        self.output_dims = output_dim
        self.feature_dims = feature_dims
        self.in_slices = [0] + list(accumulate(add, self.input_dims))
        self.out_slices = [0] + list(accumulate(add, self.output_dims))

        if freeze:
            self.generationModel.freeze()
            self.cost_encoder.freeze()


In [76]:
model_name = "MLP_MoE_R1_Two_noDropout"
model_name2 = "MLP_MoE_R5_Two_noTeacher_noDropoutInDecoder_v3_Reduced"

featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }

in_slice = [phase_dim, pose_dim, cost_dim]
in_slice2 = [phase_dim, pose_dim2, cost_dim]

out_slice = [phase_dim, config["k"], cost_dim]

temp = MLP_v2(config=config, dimensions=[pose_dim])
temp2 = MLP_v2(config=config, dimensions=[pose_dim2])

pose_encoder = temp
pose_encoder2 = temp2
pose_encoder.encoder.load_state_dict(ae1.encoder.state_dict())
pose_encoder.decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder2.encoder.load_state_dict(ae2.encoder.state_dict())
pose_encoder2.decoder.load_state_dict(ae2.decoder.state_dict())


middle_layer = torch.nn.Sequential()
model1 = MotionGenerationModel(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=model_name
                                   )

In [64]:
model2 = MotionGenerationModel(config=config, Model=MoE, pose_autoencoder=pose_encoder2, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=model_name2
                                   )

In [77]:
fit(model1, name=model_name, version="no_teacher", MAX_EPOCHS=100)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 454 K 
1 | middle_layer     | Sequential | 0     
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 2.4 M 
------------------------------------------------
2.9 M     Trainable params
0         Non-trainable params
2.9 M     Total params
11.755    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f830f839520> for <torch.optim.swa_utils.SWALR object at 0x7f83269f1df0>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.04317111149430275,
 'ptl/test_min_pos_cost': -0.007063120603561401,
 'ptl/test_min_rot_cost': 0.21088065207004547,
 'ptl/test_min_target_pos_cost': -0.006668483838438988,
 'ptl/test_min_target_rot_cost': 0.2455190271139145,
 'ptl/test_pos_loss': 0.00013738426787313074,
 'ptl/test_rot_loss': 8.466471672058105,
 'ptl/test_sum_pos_cost': -0.04155856743454933,
 'ptl/test_sum_rot_cost': 2.7537107467651367,
 'ptl/test_sum_target_pos_cost': -0.063622385263443,
 'ptl/test_sum_target_rot_cost': 3.047912359237671,
 'test_loss': 0.08346924930810928}
--------------------------------------------------------------------------------


In [65]:
model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

fit(model2, name=model_name2, version="AE_ONLY_v3", MAX_EPOCHS=100)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 460 K 
1 | middle_layer     | Sequential | 0     
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 2.4 M 
------------------------------------------------
2.9 M     Trainable params
0         Non-trainable params
2.9 M     Total params
11.780    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f8326ada4c0> for <torch.optim.swa_utils.SWALR object at 0x7f83c8234ac0>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.03366383537650108,
 'ptl/test_min_pos_cost': -0.0009536821162328124,
 'ptl/test_min_rot_cost': 0.338715523481369,
 'ptl/test_min_target_pos_cost': -0.0011594019597396255,
 'ptl/test_min_target_rot_cost': 0.3298649489879608,
 'ptl/test_pos_loss': 0.00017047548317350447,
 'ptl/test_rot_loss': 2.639564275741577,
 'ptl/test_sum_pos_cost': 0.007019455078989267,
 'ptl/test_sum_rot_cost': 4.282515525817871,
 'ptl/test_sum_target_pos_cost': 0.001460934174247086,
 'ptl/test_sum_target_rot_cost': 4.010224342346191,
 'test_loss': 0.05840516462922096}
--------------------------------------------------------------------------------


In [66]:
clean_checkpoints(os.path.join(MODEL_PATH,model_name))
clean_checkpoints(os.path.join(MODEL_PATH,model_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)

In [78]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+model_name)
# generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,
#                    template_path="/home/nuoc/Documents/MEX/False_1_0.json",
#                    output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+model_name2)


torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])


# MLP MIX - Z as input

In [29]:
mlpmix_name = "MLPMIX-Single-R1-Z-Input"
mlpmix = MLP_MIX(config=config, input_dims=[pose_dim])

mlpmix.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
mlpmix.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = mlpmix.active_models[0]
middle_layer = mlpmix.cluster_model

model1 = MotionGenerationModel_v4(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=mlpmix_name
                                   )

In [30]:
fit(model1, mlpmix_name, version="0.2",MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 454 K 
1 | middle_layer     | Sequential | 131 K 
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 2.4 M 
------------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.281    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f568cce55e0> for <torch.optim.swa_utils.SWALR object at 0x7f568c977850>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.057931140065193176,
 'ptl/test_min_pos_cost': -0.004734804853796959,
 'ptl/test_min_rot_cost': 0.21398384869098663,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.011098526418209076,
 'ptl/test_rot_loss': 0.05978814885020256,
 'ptl/test_sum_pos_cost': -0.02137945406138897,
 'ptl/test_sum_rot_cost': 2.821186065673828,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.11007595807313919}
--------------------------------------------------------------------------------


In [31]:
mlpmix_name = "MLPMIX-Single-R2-Z-Input-Reduced"
mlpmix = MLP_MIX(config=config, input_dims=[pose_dim2])

mlpmix.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
mlpmix.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = mlpmix.active_models[0]
middle_layer = mlpmix.cluster_model
middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MotionGenerationModel_v4(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=mlpmix_name
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

<All keys matched successfully>

In [32]:

fit(model2, mlpmix_name, version="0.2", MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 460 K 
1 | middle_layer     | Sequential | 131 K 
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 2.4 M 
------------------------------------------------
2.9 M     Trainable params
131 K     Non-trainable params
3.1 M     Total params
12.306    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f562caec8b0> for <torch.optim.swa_utils.SWALR object at 0x7f55ee8828b0>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.0726909190416336,
 'ptl/test_min_pos_cost': -0.002268813783302903,
 'ptl/test_min_rot_cost': 0.3286762833595276,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.017275160178542137,
 'ptl/test_rot_loss': 0.0653780847787857,
 'ptl/test_sum_pos_cost': -0.007123080547899008,
 'ptl/test_sum_rot_cost': 4.181217193603516,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.09389448165893555}
--------------------------------------------------------------------------------


In [33]:
clean_checkpoints(os.path.join(MODEL_PATH,"MLPMIX-Single-R1-Z-Input"))
clean_checkpoints(os.path.join(MODEL_PATH,mlpmix_name))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [34]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/MLPMIX-Single-R1-Z-Input")
# generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,
#                    template_path="/home/nuoc/Documents/MEX/False_1_0.json",
#                    output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/MLPMIX-Single-R2-Z-Input-Reduced")

torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])


# MLP MIX - Z Concat

In [35]:
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"]*2 + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"]*2 + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
mlpmix_name = "MLPMIX-Single-R1-Z-Concat"
mlpmix = MLP_MIX(config=config, input_dims=[pose_dim])

mlpmix.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
mlpmix.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = mlpmix.active_models[0]
middle_layer = mlpmix.cluster_model

model1 = MotionGenerationModel_Concat(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=mlpmix_name
                                   )

In [36]:
fit(model1, mlpmix_name, version="0.2", MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 454 K 
1 | middle_layer     | Sequential | 131 K 
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 3.0 M 
------------------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.378    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f562ca5aa90> for <torch.optim.swa_utils.SWALR object at 0x7f562c9dd400>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.11958379298448563,
 'ptl/test_min_pos_cost': -0.005060994066298008,
 'ptl/test_min_rot_cost': 0.20295865833759308,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.01100150402635336,
 'ptl/test_rot_loss': 0.058779697865247726,
 'ptl/test_sum_pos_cost': -0.0016244666185230017,
 'ptl/test_sum_rot_cost': 2.7480456829071045,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.1672605574131012}
--------------------------------------------------------------------------------


In [37]:
mlpmix_name = "MLPMIX-Single-R2-Z-Concat-Reduced"
mlpmix = MLP_MIX(config=config, input_dims=[pose_dim2])

mlpmix.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
mlpmix.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = mlpmix.active_models[0]
middle_layer = mlpmix.cluster_model
middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MotionGenerationModel_Concat(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=mlpmix_name
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

<All keys matched successfully>

In [38]:

fit(model2, mlpmix_name, version="0.2", MAX_EPOCHS=100)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 460 K 
1 | middle_layer     | Sequential | 131 K 
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | MoE        | 3.0 M 
------------------------------------------------
3.5 M     Trainable params
131 K     Non-trainable params
3.6 M     Total params
14.403    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f55f0e0e3d0> for <torch.optim.swa_utils.SWALR object at 0x7f562c79c880>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.1670786738395691,
 'ptl/test_min_pos_cost': -0.0012020673602819443,
 'ptl/test_min_rot_cost': 0.31904318928718567,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.020937127992510796,
 'ptl/test_rot_loss': 0.07685936987400055,
 'ptl/test_sum_pos_cost': 0.008795861154794693,
 'ptl/test_sum_rot_cost': 4.102316856384277,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.2700470983982086}
--------------------------------------------------------------------------------


In [39]:
clean_checkpoints(os.path.join(MODEL_PATH,"MLPMIX-Single-R1-Z-Concat"))
clean_checkpoints(os.path.join(MODEL_PATH,mlpmix_name))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [40]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/MLPMIX-Single-R1-Z-Concat")
generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,
                   template_path="/home/nuoc/Documents/MEX/False_1_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+mlpmix_name)


torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])


# RBF - Z as input

In [41]:
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
rbf_name1 = "RBF-Single-R1-Z-Input"
rbf = RBF(config=config, input_dims=[pose_dim])

rbf.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
rbf.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = rbf.active_models[0]
middle_layer = rbf.cluster_model

model1 = MotionGenerationModel_v4(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=rbf_name1
                                   )

In [42]:
fit(model1, rbf_name1, version="0.2", MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 454 K 
1 | middle_layer     | RBF_Layer | 65.8 K
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 2.4 M 
-----------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
12.018    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f568cdd7a30> for <torch.optim.swa_utils.SWALR object at 0x7f568ce70550>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.09364240616559982,
 'ptl/test_min_pos_cost': -0.0053687128238379955,
 'ptl/test_min_rot_cost': 0.1818535178899765,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.020856985822319984,
 'ptl/test_rot_loss': 0.07930824160575867,
 'ptl/test_sum_pos_cost': 0.008359869010746479,
 'ptl/test_sum_rot_cost': 2.6449549198150635,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.1203242689371109}
--------------------------------------------------------------------------------


In [43]:
rbf_name2 = "RBF-Single-R2-Z-Input-Reduced"
rbf = RBF(config=config, input_dims=[pose_dim2])

rbf.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
rbf.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = rbf.active_models[0]
middle_layer = rbf.cluster_model
middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MotionGenerationModel_v4(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=rbf_name2
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

<All keys matched successfully>

In [44]:
fit(model2, rbf_name2, version="0.2", MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 460 K 
1 | middle_layer     | RBF_Layer | 65.8 K
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 2.4 M 
-----------------------------------------------
2.9 M     Trainable params
65.8 K    Non-trainable params
3.0 M     Total params
12.043    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f562c9ab670> for <torch.optim.swa_utils.SWALR object at 0x7f562c9a5f10>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.10660167038440704,
 'ptl/test_min_pos_cost': 0.0022916956804692745,
 'ptl/test_min_rot_cost': 0.3098534047603607,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.024783343076705933,
 'ptl/test_rot_loss': 0.09240834414958954,
 'ptl/test_sum_pos_cost': 0.077973872423172,
 'ptl/test_sum_rot_cost': 4.071671962738037,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.16455666720867157}
--------------------------------------------------------------------------------


In [45]:
clean_checkpoints(os.path.join(MODEL_PATH,rbf_name1))
clean_checkpoints(os.path.join(MODEL_PATH,rbf_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [46]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+rbf_name1)
generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,
                   template_path="/home/nuoc/Documents/MEX/False_1_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+rbf_name2)


torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])


# RBF - Z Concat

In [47]:
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"]*2 + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"]*2 + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
rbf_name1 = "RBF-Single-R1-Z-Concat"
rbf = RBF(config=config, input_dims=[pose_dim])

rbf.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
rbf.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = rbf.active_models[0]
middle_layer = rbf.cluster_model

model1 = MotionGenerationModel_Concat(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=rbf_name1
                                   )

In [49]:
fit(model1, rbf_name1, version="0.2", MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 454 K 
1 | middle_layer     | RBF_Layer | 65.8 K
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 3.0 M 
-----------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
14.115    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f568cddbd60> for <torch.optim.swa_utils.SWALR object at 0x7f568d11b550>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.10979676246643066,
 'ptl/test_min_pos_cost': -0.0033021767158061266,
 'ptl/test_min_rot_cost': 0.207255020737648,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.009942011907696724,
 'ptl/test_rot_loss': 0.05450308322906494,
 'ptl/test_sum_pos_cost': 0.009184036403894424,
 'ptl/test_sum_rot_cost': 2.807267904281616,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.17189908027648926}
--------------------------------------------------------------------------------


In [50]:
rbf_name2 = "RBF-Single-R2-Z-Concat-Reduced"
rbf = RBF(config=config, input_dims=[pose_dim2])

rbf.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
rbf.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = rbf.active_models[0]
middle_layer = rbf.cluster_model
middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MotionGenerationModel_Concat(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=rbf_name2
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

<All keys matched successfully>

In [51]:

fit(model2, rbf_name2, version="0.2", MAX_EPOCHS=100)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 460 K 
1 | middle_layer     | RBF_Layer | 65.8 K
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 3.0 M 
-----------------------------------------------
3.5 M     Trainable params
65.8 K    Non-trainable params
3.5 M     Total params
14.140    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f562c76aca0> for <torch.optim.swa_utils.SWALR object at 0x7f55ec536610>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.15703630447387695,
 'ptl/test_min_pos_cost': -0.0006652036099694669,
 'ptl/test_min_rot_cost': 0.3239959478378296,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.020156705752015114,
 'ptl/test_rot_loss': 0.07347622513771057,
 'ptl/test_sum_pos_cost': 0.030438141897320747,
 'ptl/test_sum_rot_cost': 4.198546409606934,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.175152987241745}
--------------------------------------------------------------------------------


In [52]:
clean_checkpoints(os.path.join(MODEL_PATH,rbf_name1))
clean_checkpoints(os.path.join(MODEL_PATH,rbf_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [53]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+rbf_name1)
generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,
                   template_path="/home/nuoc/Documents/MEX/False_1_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+rbf_name2)



torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])


# VAE - Z as input

In [54]:
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
vae_name1 = "VAE-Single-R1-Z-Input"
vae = VAE(config=config, input_dims=[pose_dim])

vae.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
vae.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = vae.active_models[0]
middle_layer = vae.cluster_model

model1 = MotionGenerationModel_VAE(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=vae_name1
                                   )

In [55]:
fit(model1, vae_name1, MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 454 K 
1 | middle_layer     | VAE_Layer | 131 K 
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 2.4 M 
-----------------------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.281    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f55e9cdc760> for <torch.optim.swa_utils.SWALR object at 0x7f569788db50>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_kl_loss': 0.004071646369993687,
 'ptl/test_loss': 0.08888862282037735,
 'ptl/test_min_pos_cost': -0.004451391752809286,
 'ptl/test_min_rot_cost': 0.2351878434419632,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.019994894042611122,
 'ptl/test_recon_loss': 0.08481697738170624,
 'ptl/test_rot_loss': 0.0779380202293396,
 'ptl/test_sum_pos_cost': -0.024823816493153572,
 'ptl/test_sum_rot_cost': 2.9984066486358643,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.12005238980054855}
--------------------------------------------------------------------------------


In [56]:
vae_name2 = "VAE-Single-R2-Z-Input-Reduced"
vae = VAE(config=config, input_dims=[pose_dim2])

vae.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
vae.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = vae.active_models[0]
middle_layer = vae.cluster_model
middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MotionGenerationModel_VAE(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=vae_name2
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

<All keys matched successfully>

In [57]:
fit(model2, vae_name2, MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 460 K 
1 | middle_layer     | VAE_Layer | 131 K 
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 2.4 M 
-----------------------------------------------
2.9 M     Trainable params
131 K     Non-trainable params
3.1 M     Total params
12.306    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f5624ad97f0> for <torch.optim.swa_utils.SWALR object at 0x7f568c7ce190>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_kl_loss': 0.002039763145148754,
 'ptl/test_loss': 0.11052046716213226,
 'ptl/test_min_pos_cost': 0.001397526590153575,
 'ptl/test_min_rot_cost': 0.32723695039749146,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.02676679566502571,
 'ptl/test_recon_loss': 0.10848070681095123,
 'ptl/test_rot_loss': 0.09787915647029877,
 'ptl/test_sum_pos_cost': 0.05337149649858475,
 'ptl/test_sum_rot_cost': 4.250061988830566,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.1679312139749527}
--------------------------------------------------------------------------------


In [58]:
clean_checkpoints(os.path.join(MODEL_PATH,vae_name1))
clean_checkpoints(os.path.join(MODEL_PATH,vae_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [59]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,use_vae=True,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+vae_name1)
generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,use_vae=True,
                   template_path="/home/nuoc/Documents/MEX/False_1_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+vae_name2)


torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])


# VAE - Z Concat

In [62]:
featureDim = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"]*2 + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims["pos"],
    "rot_dim":feature_dims["rotMat2"],
    "vel_dim":feature_dims["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
featureDim2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim2,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"]*2 + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim,
    "pos_dim":feature_dims2["pos"],
    "rot_dim":feature_dims2["rotMat2"],
    "vel_dim":feature_dims2["velocity"],
    "posCost":feature_dims["posCost"],
    "rotCost":feature_dims["rotCost"]
    }
vae_name1 = "VAE-Single-R1-Z-Concat"
vae = VAE(config=config, input_dims=[pose_dim])

vae.active_models[0].encoder.load_state_dict(ae1.encoder.state_dict())
vae.active_models[0].decoder.load_state_dict(ae1.decoder.state_dict())

pose_encoder = vae.active_models[0]
middle_layer = vae.cluster_model

model1 = MotionGenerationModel_VAE_ZZ(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=t1, val_set=v1, test_set=te1,
                                 name=vae_name1
                                   )

In [63]:
fit(model1, vae_name1, MAX_EPOCHS=100)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 454 K 
1 | middle_layer     | VAE_Layer | 131 K 
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 3.0 M 
-----------------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.378    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f568c977850> for <torch.optim.swa_utils.SWALR object at 0x7f562c84ec10>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_kl_loss': 2.144406062143389e-05,
 'ptl/test_loss': 0.04409467428922653,
 'ptl/test_min_pos_cost': -0.004690519068390131,
 'ptl/test_min_rot_cost': 0.22766542434692383,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.009413277730345726,
 'ptl/test_recon_loss': 0.044073231518268585,
 'ptl/test_rot_loss': 0.05002091825008392,
 'ptl/test_sum_pos_cost': -0.02753099799156189,
 'ptl/test_sum_rot_cost': 2.9506356716156006,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.07824178040027618}
--------------------------------------------------------------------------------


In [64]:
vae_name2 = "VAE-Single-R2-Z-Concat-Reduced"
vae = VAE(config=config, input_dims=[pose_dim2])

vae.active_models[0].encoder.load_state_dict(ae2.encoder.state_dict())
vae.active_models[0].decoder.load_state_dict(ae2.decoder.state_dict())

pose_encoder = vae.active_models[0]
middle_layer = vae.cluster_model
middle_layer.load_state_dict(model1.middle_layer.state_dict())
middle_layer.requires_grad_(False)


model2 = MotionGenerationModel_VAE_ZZ(config=config, Model=MoE, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=featureDim2,
                                 input_slicers=in_slice2, output_slicers=out_slice,
                                 train_set=t2, val_set=v2, test_set=te2,
                                 name=vae_name2
                                   )

model2.generationModel.gate.load_state_dict(model1.generationModel.gate.state_dict())
model2.generationModel.load_state_dict(model1.generationModel.state_dict())

<All keys matched successfully>

In [65]:

fit(model2, vae_name2, MAX_EPOCHS=100)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type      | Params
-----------------------------------------------
0 | pose_autoencoder | MLP       | 460 K 
1 | middle_layer     | VAE_Layer | 131 K 
2 | cost_encoder     | MLP       | 36.2 K
3 | generationModel  | MoE       | 3.0 M 
-----------------------------------------------
3.5 M     Trainable params
131 K     Non-trainable params
3.6 M     Total params
14.403    Total estimated model params size (MB)


/home/nuoc/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Swapping lr_scheduler <torch.optim.lr_scheduler.StepLR object at 0x7f568c986580> for <torch.optim.swa_utils.SWALR object at 0x7f55ee969310>
  warnings.warn(*args, **kwargs)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_kl_loss': 0.00015117238217499107,
 'ptl/test_loss': 0.0905328169465065,
 'ptl/test_min_pos_cost': -0.0015516206622123718,
 'ptl/test_min_rot_cost': 0.34485024213790894,
 'ptl/test_min_target_pos_cost': nan,
 'ptl/test_min_target_rot_cost': nan,
 'ptl/test_pos_loss': 0.022057222202420235,
 'ptl/test_recon_loss': 0.09038165211677551,
 'ptl/test_rot_loss': 0.07953483611345291,
 'ptl/test_sum_pos_cost': -0.003954050596803427,
 'ptl/test_sum_rot_cost': 4.304420471191406,
 'ptl/test_sum_target_pos_cost': nan,
 'ptl/test_sum_target_rot_cost': nan,
 'test_loss': 0.1030849739909172}
--------------------------------------------------------------------------------


In [66]:
clean_checkpoints(os.path.join(MODEL_PATH,vae_name1))
clean_checkpoints(os.path.join(MODEL_PATH,vae_name2))

model1.save_checkpoint(best_val_loss=0.001)
model2.save_checkpoint(best_val_loss=0.001)


In [67]:
generate_animation(model=model1, test_set=te11, feature_dims=feature_dims,use_vae=True,
                   template_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+vae_name1)
generate_animation(model=model2, test_set=te22, feature_dims=feature_dims2,use_vae=True,
                   template_path="/home/nuoc/Documents/MEX/False_1_0.json",
                   output_path="/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/Replay/"+vae_name2)





torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186]) torch.Size([5, 299, 93])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])
torch.Size([5, 299, 96]) torch.Size([5, 299, 192]) torch.Size([5, 299, 96])


In [72]:
def setVec3(struct, vec):
    struct["x"] = vec[0].item()
    struct["y"] = vec[1].item()
    struct["z"] = vec[2].item()

def setVec6(struct, vec):
    for r, cell in enumerate(["x", "y", "z"]):
        for col, column in enumerate(["c0", "c1"]):
            struct[column][cell] = vec[r, col].item()

def insert_pos(template,
               positions=None, rotations=None, velocity=None,
               tPos=None, tRot=None, name="Replay"):
    shape = positions.shape
    for c in range(shape[0]):
        for f in range(shape[1]):
            t = 0
            for j in range(shape[2]):
                jo = template["frames"][f]["joints"][j]
                if positions is not None:
                    setVec3(jo["position"], positions[c,f,j])
                if rotations is not None:
                    setVec3(jo["velocity"], velocity[c,f,j])
                if velocity is not None:
                    setVec6(jo["rotMat"], rotations[c,f,j])


                if jo["key"]:
                    if tPos is not None:
                        setVec3(jo["cost"]["TargetPosition"], tPos[c,f,t])
                    if tRot is not None:
                        setVec6(jo["cost"]["TargetRotation"], tRot[c,f,t])
                    t+=1
        with open("{}_{}.json".format(name, c), "w") as f:
            js.dump(template, f)

def generate_animation(model, test_set, feature_dims, template_path, output_path, use_vae=False, n=5):
    idx = np.arange(n)
    with torch.no_grad():
        model.eval()
        model.cpu()
        model.pose_autoencoder.encoder.eval()
        model.pose_autoencoder.decoder.eval()
        model.generationModel.eval()

        x = torch.stack([test_set[i][0] for i in idx])
        y = torch.stack([test_set[i][1] for i in idx])
        shape = x.shape
        x = x.view(-1, shape[-1])

        x.cpu()
        if use_vae:
            out, z, mu, logvar = model(x)
        else:
            out = model(x)
        x_c = torch.cat(out,dim=1).detach()
        generated = x_c
        generated = generated.view(shape)

    phase= feature_dims["phase_vec_l2"]
    toPosDim = phase+feature_dims["pos"]
    toRotDim = toPosDim + feature_dims["rotMat2"]
    toVelDim = toRotDim + feature_dims["velocity"]

    gPos = generated[:, :, phase:toPosDim]
    gRot = generated[:, :, toPosDim:toRotDim]
    gVel = generated[:, :, toRotDim:toVelDim]

    oPos = y[:, :, phase:toPosDim]
    oRot = y[:, :, toPosDim:toRotDim]
    oVel = y[:, :, toRotDim:toVelDim]

    tPos = y[:, :, -target_dim:-target_dim+3*4]
    tRot = y[:, :, -target_dim+3*4:]

    print(gPos.shape, gRot.shape, gVel.shape)
    print(oPos.shape, oRot.shape, oVel.shape)

    clip_length = gPos.shape[1]
    gPos_r = gPos.reshape((n, clip_length, -1, 3))
    gRot_r = gRot.reshape((n, clip_length, -1, 3, 2))
    gVel_r = gVel.reshape((n, clip_length, -1, 3))

    oPos_r = oPos.reshape((n, clip_length, -1, 3))
    oRot_r = oRot.reshape((n, clip_length, -1, 3, 2))
    oVel_r = oVel.reshape((n, clip_length, -1, 3))

    tPos_r = tPos.reshape((n, clip_length, -1, 3))
    tRot_r = tRot.reshape((n, clip_length, -1, 3, 3))

    template = js.load(open(template_path))

    insert_pos(template, oPos_r, oRot_r, oVel_r, tPos_r, tRot_r,output_path+"_O")
    insert_pos(template, gPos_r, gRot_r, gVel_r, tPos_r, tRot_r,output_path+"_G")
